# Orquestador de ventas 360: resumen
Diseño esto para, partiendo de la demanda desagregada: 
- Capturar contratos
- Descargar xlsx de sistemas SAI, CAMUNDA, SAGI, ZOHO y PISP. 
- Consultar exceles internos
- Organizar las órdenes descargadas
- Generar los pickle de las órdenes
Todo lo anterior es la información mínima indispensable para poder generar información de ventas, que será la referencia interna sobre la que desplegaremos el alimentado de reportes y la generación de reglas para dar retro al usuario.

In [1]:
# Importa librerías y chromedriver
import sys
import os
import glob
import yaml
import pandas as pd
from datetime import datetime
# Define the root and script directory
folder_root = os.getcwd()  # Get current directory (where Orquestación.ipynb is)
script_folder = os.path.join(folder_root, "Scripts")  # Path to 'Scripts'
# Ensure the script folder is added to sys.path
if script_folder not in sys.path:
    sys.path.append(script_folder)
Libreria_SAI = os.path.join(script_folder, "Libreria_SAI")
if Libreria_SAI not in sys.path:
    sys.path.append(Libreria_SAI)
Libreria_comunes = os.path.join(script_folder, "Libreria_comunes")
if Libreria_comunes not in sys.path:
    sys.path.append(Libreria_comunes)
Libreria_camunda = os.path.join(script_folder, "Libreria_camunda")
if Libreria_camunda not in sys.path:
    sys.path.append(Libreria_camunda)
Libreria_SAGI = os.path.join(script_folder, "Libreria_SAGI")
if Libreria_SAGI not in sys.path:
    sys.path.append(Libreria_SAGI)

Libreria_PREI = os.path.join(script_folder, "Libreria_PREI")
if Libreria_PREI not in sys.path:
    sys.path.append(Libreria_PREI)
from chrome_driver_load import load_chrome


## SAI
Esta consola de jupyter lab es se va a encargar de: 
- Descargar xlsx de sistemas SAI

In [ ]:
# Descargar SAI
from SAI_download import SAI_download
# Define your download directory
download_directory = os.path.join(folder_root,'Implementación', 'Descarga de bases')
#print(download_directory)
driver = load_chrome(download_directory)
if driver:
    # Execute the SAI download process
    SAI_download(driver, "EPH -161215-NS9","Epharma2021" )
    # Optionally, close the driver when done
    driver.quit()

## Camunda - Órdenes

In [ ]:
from CAMUNDA_login import SAI_camunda

download_directory = os.path.join(folder_root,'Implementación', 'Descarga de bases')

# Initialize the Chrome driver with your custom settings
driver = load_chrome(download_directory)

if driver:
    # Execute the Camunda login and initial automation
    SAI_camunda(driver, "ArmandoJimenez", "N29f6Mwif")
    driver.quit()

## Camunda - Remisiones

In [ ]:
## Aquí debemos pasar el CSV maestro de la extracción, si no existe skip function entera. 
## La dirección del output
## El webdriver es el mismo

## SAGI


In [ ]:
from SAGI_download import SAGI_download
# Initialize Chrome with your custom settings
download_directory = os.path.join(folder_root,'Implementación', 'Descarga de bases')
driver = load_chrome(download_directory)
if driver:
    # Call the function with your login credentials (username and password)
    SAGI_download(driver, "EPH161215NS9", "Pharmas21", download_directory)
    driver.quit()

In [ ]:
from SAGI_Join_2023_2024 import join_SAGI_files, worksheet_to_df

json_key = os.path.join(folder_root, 'Implementación', 'Key.json')
file2023_2024 = os.path.join(folder_root, 'Implementación', 'Descarga de bases', "03 07 2023-2024.xlsx")
file_2024 = os.path.join(folder_root, 'Implementación', 'Descarga de bases', "03 07 2024.xlsx")
output_joined_file = os.path.join(folder_root, 'Implementación', "03 07 2024 ESTATUS_SAGI.xlsx")

join_SAGI_files(json_key, file2023_2024, file_2024, output_joined_file)

## PISP (PREI)


In [2]:
# Carga librerías y define el año a descargar.
from PREI_downloader import PREI_downloader

user = '0000150462'
password = 'EPH161215NS9'

year = input() 
dates = os.path.join(folder_root,'Implementación', 'PREI', f"{year}_dates.xlsx")
temp_folder = os.path.join(folder_root,'Implementación', 'PREI', '{year} Temporal')

# Allowed years
valid_years = {"2023", "2024", "2025"}

# Loop until a valid year is entered
while True:
    year = input("Enter a year (2023, 2024, 2025): ").strip()
    if year in valid_years:
        break
    else:
        print("Invalid year. Please try again.")

# Create paths using f-strings to substitute the year
dates = os.path.join(folder_root, 'Implementación', 'PREI', f"{year}_dates.xlsx")
temp_folder = os.path.join(folder_root, 'Implementación', 'PREI', f"{year} Temporal")
final_folder = os.path.join(folder_root, 'Implementación', 'PREI', f"{year} Final")
# Check if the Excel file exists
def check_if_exists(element):
    if os.path.exists(element):
        print(f"The file {os.path.basename(element)} exists.")
    else:
        print(f"The file {os.path.basename(element)} does not exist.")
check_if_exists(dates)
check_if_exists(temp_folder)
check_if_exists(final_folder)


 2024
Enter a year (2023, 2024, 2025):  2024


The file 2024_dates.xlsx exists.
The file 2024 Temporal exists.
The file 2024 Final exists.


In [3]:
#Descargar XLS del PREI
driver = load_chrome(temp_folder)
if driver:
    # Call the function with your login credentialstemp_folder(username and password)
    PREI_downloader(driver, user, password, temp_folder, dates)
    driver.quit()

Processing row 1: Date START = 16/08/2024, Date END = 31/08/2024
Calling input_date for fecha_inicial...
Date passed: 16/08/2024
Field cleared successfully on attempt 1
Attempting to input date: 16/08/2024
fecha_inicial set successfully.
Calling input_date for fecha_final...
Date passed: 31/08/2024
Field cleared successfully on attempt 1
Attempting to input date: 31/08/2024
fecha_final set successfully.
Clicking 'buscar' button...
Dates 16/08/2024 to 31/08/2024 were processed
Moving to the next set.
Processing row 2: Date START = 01/09/2024, Date END = 15/09/2024
Calling input_date for fecha_inicial...
Date passed: 01/09/2024
Field cleared successfully on attempt 1
Attempting to input date: 01/09/2024
fecha_inicial set successfully.
Calling input_date for fecha_final...
Date passed: 15/09/2024
Field cleared successfully on attempt 1
Attempting to input date: 15/09/2024
fecha_final set successfully.
Clicking 'buscar' button...
Dates 01/09/2024 to 15/09/2024 were processed
Moving to the 

In [4]:
# Mover arhivos descargados a su carpeta final
from PREI_merger_and_audit import move_files
move_files(temp_folder, final_folder)

Moved file: [FacturaVsCR][0000150462][21-11-2024][01-12-2024].xls to 2024 Final
Moved file: [FacturaVsCR][0000150462][16-08-2024][31-08-2024].xls to 2024 Final
Moved file: [FacturaVsCR][0000150462][16-04-2024][30-04-2024].xls to 2024 Final
Moved file: [FacturaVsCR][0000150462][16-01-2024][31-01-2024].xls to 2024 Final
Moved file: [FacturaVsCR][0000150462][01-07-2024][15-07-2024].xls to 2024 Final
Moved file: [FacturaVsCR][0000150462][11-12-2024][15-12-2024].xls to 2024 Final
Moved file: [FacturaVsCR][0000150462][17-02-2024][29-02-2024].xls to 2024 Final
Moved file: [FacturaVsCR][0000150462][01-08-2024][15-08-2024].xls to 2024 Final
Moved file: [FacturaVsCR][0000150462][01-04-2024][15-04-2024].xls to 2024 Final
Moved file: [FacturaVsCR][0000150462][01-10-2024][31-10-2024].xls to 2024 Final
Moved file: [FacturaVsCR][0000150462][01-11-2024][20-11-2024].xls to 2024 Final
Moved file: [FacturaVsCR][0000150462][16-07-2024][31-07-2024].xls to 2024 Final
Moved file: [FacturaVsCR][0000150462][01

In [3]:
# Genera un solo excel con la información de todos
from PREI_merger_and_audit import merge_files
merge_files(temp_folder, final_folder)


Abriendo los exceles fuentes:
Combinando exceles en la carpeta: {'2024 Final'} 

Duplicated rows before merging:
     Documento                          Folio Fiscal       Fecha Factura  \
1     34132303  774451F0-5205-43C2-A1A2-FF3D64E37D8A 2024-11-26 08:56:33   
2     34132303  774451F0-5205-43C2-A1A2-FF3D64E37D8A 2024-11-26 08:56:33   
3     34132305  96AEDC73-B5AB-4AE3-9D96-0BECD52C5846 2024-11-26 08:45:31   
4     34132305  96AEDC73-B5AB-4AE3-9D96-0BECD52C5846 2024-11-26 08:45:31   
5     34132308  AD20FC95-46AE-450E-8D66-262C04F4D781 2024-11-26 08:55:02   
..         ...                                   ...                 ...   
773   34335645  E74E7BCF-1DA5-4D08-83DE-5DDBDEDF13B3 2024-12-26 09:23:26   
774   34335646  DC20D500-6B81-4D80-BBCE-4898DF4979A2 2024-12-26 09:19:50   
775   34335646  DC20D500-6B81-4D80-BBCE-4898DF4979A2 2024-12-26 09:19:50   
776   34335647  1B3B2F69-F1F0-4C38-9AB6-B11D5ACC55F9 2024-12-26 09:17:55   
777   34335647  1B3B2F69-F1F0-4C38-9AB6-B11D5ACC55

In [4]:
# Audita los archivos
from PREI_merger_and_audit import audit
audit(final_folder, year)

Data extraction and processing complete. 2024 audit.xlsx has been updated.


In [5]:

import os
import glob
import pandas as pd
from datetime import datetime
#Une todo

# Define expected headers (adjust as needed)
expected_headers = ["Documento", "Folio Fiscal", "Fecha Factura", "Importe", 
                    "Fecha Carga", "Unidad Negocio", "Contra Recibo", "Estado C.R.", "Fecha"]

# List to collect DataFrames from each recent xlsx file
dataframes = []

# Loop through each valid year
for year in valid_years:
    final_folder = os.path.join(folder_root, 'Implementación', 'PREI', f"{year} Final")
    # Find all xlsx files in the final folder
    xlsx_files = glob.glob(os.path.join(final_folder, "*.xlsx"))
    if not xlsx_files:
        print(f"No xlsx files found in {final_folder}")
        continue

    # Find the most recent xlsx file based on creation time
    recent_file = max(xlsx_files, key=os.path.getctime)
    creation_time = os.path.getctime(recent_file)
    creation_dt = datetime.fromtimestamp(creation_time)
    print(f"Recent file in {final_folder}: {os.path.basename(recent_file)} (Created: {creation_dt.strftime('%Y-%m-%d %H:%M:%S')})")

    # Load the recent file
    try:
        df = pd.read_excel(recent_file, engine='openpyxl')
    except Exception as e:
        print(f"Error reading {recent_file}: {e}")
        continue

    # Check if the file's headers match the expected headers
    if list(df.columns) != expected_headers:
        print(f"Headers in {os.path.basename(recent_file)} do not match expected headers. Skipping this file.")
        continue

    dataframes.append(df)

# Merge all valid DataFrames into one master DataFrame and save
if dataframes:
    master_df = pd.concat(dataframes, ignore_index=True)
    master_file = os.path.join(folder_root, 'Implementación', 'PREI', "PREI.xlsx")
    master_df.to_excel(master_file, index=False, engine='openpyxl')
    print(f"Master file saved as: {master_file}")
else:
    print("No valid files found to merge.")

Recent file in /Users/armjorge/Library/CloudStorage/Dropbox/3. Armando Cuaxospa/Licitaciones/Licitaciones 2025/E115 360/Implementación/PREI/2023 Final: 2025 03 13 PISP 10h.xlsx (Created: 2025-03-13 10:06:26)
Recent file in /Users/armjorge/Library/CloudStorage/Dropbox/3. Armando Cuaxospa/Licitaciones/Licitaciones 2025/E115 360/Implementación/PREI/2024 Final: 2024 audit.xlsx (Created: 2025-03-13 10:52:51)
Headers in 2024 audit.xlsx do not match expected headers. Skipping this file.
Recent file in /Users/armjorge/Library/CloudStorage/Dropbox/3. Armando Cuaxospa/Licitaciones/Licitaciones 2025/E115 360/Implementación/PREI/2025 Final: 2025 audit.xlsx (Created: 2025-03-13 10:15:38)
Headers in 2025 audit.xlsx do not match expected headers. Skipping this file.
Master file saved as: /Users/armjorge/Library/CloudStorage/Dropbox/3. Armando Cuaxospa/Licitaciones/Licitaciones 2025/E115 360/Implementación/PREI/PREI.xlsx
